<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -qU langchain openai faiss-cpu sentence_transformers chromadb

In [ ]:
# from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import openai
key = 'key'
llm = ChatOpenAI(
    openai_api_key=key,
    temperature=0,
    model_name="gpt-3.5-turbo"
)

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings


In [ ]:
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader

two_web_links = ["https://www.databricks.com/","https://help.databricks.com","https://databricks.com/try-databricks","https://help.databricks.com/s/","https://docs.databricks.com","https://kb.databricks.com/","http://docs.databricks.com/getting-started/index.html","http://docs.databricks.com/introduction/index.html","http://docs.databricks.com/getting-started/tutorials/index.html","http://docs.databricks.com/release-notes/index.html","http://docs.databricks.com/ingestion/index.html","http://docs.databricks.com/exploratory-data-analysis/index.html","http://docs.databricks.com/data-preparation/index.html","http://docs.databricks.com/data-sharing/index.html","http://docs.databricks.com/marketplace/index.html","http://docs.databricks.com/workspace-index.html","http://docs.databricks.com/machine-learning/index.html","http://docs.databricks.com/sql/index.html","http://docs.databricks.com/delta/index.html","http://docs.databricks.com/dev-tools/index.html","http://docs.databricks.com/integrations/index.html","http://docs.databricks.com/administration-guide/index.html","http://docs.databricks.com/security/index.html","http://docs.databricks.com/data-governance/index.html","http://docs.databricks.com/lakehouse-architecture/index.html","http://docs.databricks.com/reference/api.html","http://docs.databricks.com/resources/index.html","http://docs.databricks.com/whats-coming.html","http://docs.databricks.com/archive/index.html","http://docs.databricks.com/lakehouse/index.html","http://docs.databricks.com/getting-started/quick-start.html","http://docs.databricks.com/getting-started/etl-quick-start.html","http://docs.databricks.com/getting-started/lakehouse-e2e.html","http://docs.databricks.com/getting-started/free-training.html","http://docs.databricks.com/sql/language-manual/index.html","http://docs.databricks.com/error-messages/index.html","http://www.apache.org/","https://databricks.com/privacy-policy","https://databricks.com/terms-of-use"]
web_links = ["https://www.pinecone.io/learn/vector-embeddings/", "https://www.featureform.com/post/the-definitive-guide-to-embeddings"]
loader = WebBaseLoader(web_links)
documents = loader.load()

In [ ]:
from importlib.metadata import metadata
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
all_splits = text_splitter.split_documents(documents)

In [ ]:
len(all_splits)

31

In [ ]:
docs = []
for d in all_splits:
  docs.append(d.page_content)

In [ ]:
docs[0]

'What are Vector Embeddings | PineconeProductSolutionsPricingResourcesCompanyLog InSign Up FreeLearn | ArticleWhat are Vector EmbeddingsJump to section IntroductionCreating Vector EmbeddingsExample: Image Embedding with a Convolutional Neural NetworkUsing Vector EmbeddingsIntroductionVector embeddings are one of the most fascinating and useful concepts in machine learning. They are central to many NLP, recommendation, and search algorithms. If you’ve ever used things like recommendation engines, voice assistants, language translators, you’ve come across systems that rely on embeddings.ML algorithms, like most software algorithms, need numbers to work with. Sometimes we have a dataset with columns of numeric values or values that can be translated into them (ordinal, categorical, etc). Other times we come across something more abstract like an entire document of text. We create vector embeddings, which are just lists of numbers, for data like this to perform various operations with them

INDEXFLATL2

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

vectorstore = FAISS.from_documents(all_splits, embedding_function)

In [ ]:
import time
query = 'What is a vector embedding?, only use context information to answer, if there isnt anything related, reply I dont know.'
vectorstore.similarity_search_with_relevance_scores(query)[1][1]

# chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

# chat_history = []
# start = time.time()
# result = chain({"question": query, "chat_history": chat_history})
# end = time.time()
# print(end - start)

0.6022298448717652

In [ ]:
L2 = result['answer']

In [ ]:
L2

'A vector embedding is a representation of an object or concept as a vector in a multi-dimensional space. It is created by assigning specific values to each dimension of the vector based on the relationship between different objects or concepts. Vector embeddings are commonly used in various applications to capture similarities and relationships between different entities.'

INDEXIVFFLAT

In [ ]:
sentence_embeddings = model.encode(docs)
d = sentence_embeddings.shape[1]
d

768

In [ ]:
import faiss

nlist = 16  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer,d, nlist)

In [ ]:
xq = model.encode(["What is a vector embedding?"])

In [ ]:
faiss.normalize_L2(sentence_embeddings)
faiss.normalize_L2(xq)

In [ ]:
index.train(sentence_embeddings)
index.add(sentence_embeddings)

In [ ]:
%%time
k = 4
D, I = index.search(xq, k)
print(D, I)
IVF =  I[0][0]


[[1.0349531 1.0921106 1.102596  1.1371037]] [[ 7  1 20  6]]
CPU times: user 704 µs, sys: 0 ns, total: 704 µs
Wall time: 711 µs


In [ ]:
docs[I[0][0]]

'encoder contain the necessary information for the decoder to produce a result. This architecture is widely used in applications, such as machine translation and caption generation.Check out some applications you can build with vector embeddings and Pinecone.Share via:  Rajat TripathiSoftware EngineerJump to section IntroductionCreating Vector EmbeddingsExample: Image Embedding with a Convolutional Neural NetworkUsing Vector EmbeddingsPRODUCTOverviewDocumentationTrust and SecuritySOLUTIONSSearchGenerative AICustomersRESOURCESLearning CenterCommunityPinecone BlogSupport CenterSystem StatusCOMPANYAboutPartnersCareersNewsroomContactLEGALTermsPrivacyCookies© Pinecone Systems, Inc. | San Francisco, CAPinecone is a registered trademark of Pinecone Systems, Inc.'

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import time

llm = OpenAI(temperature=0.9,openai_api_key=key)
prompt = PromptTemplate(
    input_variables=["source"],
    template="Given the context ### {source}###?" + "What is a vector embedding?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
start = time.time()

# Run the chain only specifying the input variable.

IVFFLAT = chain.run(docs[IVF])
print(IVFFLAT)
end = time.time()
print(end - start)

 Vector embeddings are mappings of data into a dimensional space that enables machines to reason with it in a meaningful way. They allow machines to find patterns in data that would be difficult for humans to identify or classify. Vector embeddings can be used for tasks such as image recognition, text classification, speech recognition, and natural language processing. With Pinecone, you can use vector embeddings to build machine learning applications that unlock insights from your data. Some examples of applications you can build with vector embeddings and Pinecone include: 

- Image recognition
- Text classification
- Speech recognition
- Natural language processing
- Sentiment analysis
- Recommendation systems
- Image captioning
- Chatbot development
3.639477014541626


INDEXIVFPQ

In [ ]:
m = 4  # number of centroid IDs in final compressed vectors
bits = 4 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [ ]:
index.train(sentence_embeddings)
index.add(sentence_embeddings)
index.nprobe = 3  # align to previous IndexIVFFlat nprobe value

In [ ]:
%%time
k =4
D, I = index.search(xq, k)
print(D, I)
PQ = I[0][0]

[[1.0130075 1.0130075 1.0407193 1.0407193]] [[20 51  7 38]]
CPU times: user 1.01 ms, sys: 0 ns, total: 1.01 ms
Wall time: 917 µs


In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9,openai_api_key=key)
prompt = PromptTemplate(
    input_variables=["source"],
    template="Given the context ### {source}###?" + "What is a vector embedding?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
start = time.time()

# Run the chain only specifying the input variable.
result = chain.run(docs[PQ])
print(result)
IVFPQ = result
end = time.time()
print(end - start)



Vector embedding is a technique used in natural language processing (NLP) to represent text and images as numerical vectors so they can be used in machine learning models. It maps words, phrases, and sentences to numerical vectors of fixed size that are then used in a variety of neural networks and other machine learning models. These vectors capture the semantic meaning of the words and phrases in the text, allowing the model to understand the context of the text and create more meaningful predictions. Vector embedding techniques have been used to create models for sentiment analysis, document classification, and other NLP tasks.
2.924916982650757


In [ ]:
times = {"INDEXFLATL2": 3.978,
 "INDEXIVFFLAT": 3.015,
 "INDEXIVFPQ": 1.840}
from pprint import pprint
pprint(str(times).strip('{').strip(',').strip('}'))
answers = times = {"INDEXFLATL2": L2,
 "INDEXIVFFLAT": IVFFLAT.strip('\n'),
 "INDEXIVFPQ": IVFPQ.strip('\n')}
print('\n')
pprint(answers)

"'INDEXFLATL2': 3.978, 'INDEXIVFFLAT': 3.015, 'INDEXIVFPQ': 1.84"


NameError: ignored